# Import libraries

In [2]:
# !pip install xgboost
# !pip install sklearn

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import gc

#Load data

In [4]:
train=pd.read_table('train_data.csv', delimiter=',')
test=pd.read_table('test_data.csv', delimiter=',')

In [5]:
train=train.drop('index', axis=1) #drop index column
test=test.drop('index', axis=1)

In [6]:
train.head()

,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,31.52,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,31.52,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,31.52,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,31.52,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,31.52,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95


In [7]:
print('Train:', train.shape, "| Test:", test.shape)

Train: (375734, 245) | Test: (31354, 244)


In [8]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375734 entries, 0 to 375733
Columns: 245 entries, lat to wind-vwnd-925-2010-20
dtypes: float64(240), int64(3), object(2)
memory usage: 702.3+ MB
None


#EDA

In [9]:
#print columns with categorical data
train.select_dtypes(include='object').columns

Index(['startdate', 'climateregions__climateregion'], dtype='object')

# Data preprocessing

## Fill missing values

In [10]:
#Find the Null-containing columns  
null_cols = train.columns[train.isna().any()].tolist()
null_cols

['nmme0-tmp2m-34w__ccsm30',
 'nmme-tmp2m-56w__ccsm3',
 'nmme-prate-34w__ccsm3',
 'nmme0-prate-56w__ccsm30',
 'nmme0-prate-34w__ccsm30',
 'nmme-prate-56w__ccsm3',
 'nmme-tmp2m-34w__ccsm3',
 'ccsm30']

In [11]:
null_cols_test = test.columns[test.isna().any()].tolist()

In [12]:
#impute (fill) missing values using sklearn's Simple Imputer (fill the mean value along each column)
for col in null_cols: 
    train[col] = train[col].interpolate()
train.columns[train.isna().any()].tolist()

[]

# Feature engineering

## Encoding regions

In [13]:
#rewrite as function later
def encode_regions(train, test): 
    encoder = LabelEncoder()
    new_climate_regions = encoder.fit_transform(train['climateregions__climateregion'])
    train['climateregions__climateregion'] = new_climate_regions
    test['climateregions__climateregion'] = encoder.transform(test['climateregions__climateregion'])
    cli_reg_map = {index: new_climate_regions for index, new_climate_regions in 
                      enumerate(encoder.classes_)}
    print(cli_reg_map)
    #Check for difference climate regions between train and test sets
    train_regions = train.climateregions__climateregion.value_counts
    test_regions = train.climateregions__climateregion.value_counts
    #print(train_regions == test_regions) #train and test on same climate regions
    #train and test set cover resembling climate regions
    return train, test

In [14]:
encode_regions(train, test)
train['climateregions__climateregion'].unique()

{0: 'BSh', 1: 'BSk', 2: 'BWh', 3: 'BWk', 4: 'Cfa', 5: 'Cfb', 6: 'Csa', 7: 'Csb', 8: 'Dfa', 9: 'Dfb', 10: 'Dfc', 11: 'Dsb', 12: 'Dsc', 13: 'Dwa', 14: 'Dwb'}


array([ 0,  4,  1,  3,  2,  6,  7,  5,  9, 12, 10,  8, 11, 13, 14])

## Location encoding

Encode location using latitude and longitude, round values to 14th decimal digits.
Reference: `https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved`

In [15]:
def group_loc(train, test):
    #Scale lat and lon to 14th decimal digit
    scale = 14
    train.loc[:,'lat'] = round(train.lat, scale)
    train.loc[:,'lon'] = round(train.lon, scale)
    test.loc[:,'lat'] = round(test.lat, scale)
    test.loc[:,'lon'] = round(test.lon, scale)
    # Concatenate train and test data
    all_df = pd.concat([train, test], axis=0)
    # Create new feature
    all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
    print(f'{all_df.loc_group.nunique()} unique locations')
    
    # Split back up
    train = all_df.iloc[:len(train)]
    test = all_df.iloc[len(train):]

    #Check result
    print('Locations in train that are not in test')
    locations_train=list(train.loc_group.unique())
    locations_test=list(test.loc_group.unique())
    result_1 = list(set(locations_train).difference(locations_test))
    print(result_1)

    print('Locations in test that are not in train')
    result_2=list(set(locations_test).difference(locations_train))
    print(result_2)
    
    return train, test

In [16]:
group_loc(train, test)

514 unique locations
Locations in train that are not in test
[]
Locations in test that are not in train
[]


(        lat       lon startdate  contest-pevpr-sfc-gauss-14d__pevpr  \
 0       0.0  0.833333    9/1/14                              237.00   
 1       0.0  0.833333    9/2/14                              228.90   
 2       0.0  0.833333    9/3/14                              220.69   
 3       0.0  0.833333    9/4/14                              225.28   
 4       0.0  0.833333    9/5/14                              237.24   
 ...     ...       ...       ...                                 ...   
 375729  1.0  0.866667   8/27/16                              312.05   
 375730  1.0  0.866667   8/28/16                              305.82   
 375731  1.0  0.866667   8/29/16                              311.62   
 375732  1.0  0.866667   8/30/16                              304.54   
 375733  1.0  0.866667   8/31/16                              295.29   
 
         nmme0-tmp2m-34w__cancm30  nmme0-tmp2m-34w__cancm40  \
 0                          29.02                     31.64   
 1      

## Cyclical date conversion

In [17]:
#add season
def add_season(df):
  month_to_season = {
      1: 0,
      2: 0,
      3: 1,
      4: 1,
      5: 1,
      6: 2,
      7: 2,
      8: 2, 
      9: 3, 
      10: 3,
      11: 3,
      12: 0
  }
  df['season'] = df['month'].apply(lambda x: month_to_season[x])

In [18]:
def convert_time(df):
    df['year'] = df['DATE'].dt.year
    df['month'] = df['DATE'].dt.month
    df['day'] = df['DATE'].dt.day_of_year
#     df['week'] = df['DATE'].dt.isocalendar().week
#     df['quarter'] = df['DATE'].dt.quarter
    return df

In [19]:
from sklearn.preprocessing import FunctionTransformer

def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

def encode_cyclical(df):
    # encode the day with a period of 365
    df['day_of_year_sin'] = sin_transformer(365).fit_transform(df['day'])
    df['day_of_year_cos'] = cos_transformer(365).fit_transform(df['day'])

#     # encode the week with a period of 52
#     df['week_sin'] = sin_transformer(52).fit_transform(df['week'])
#     df['week_cos'] = cos_transformer(52).fit_transform(df['week'])

    # encode the month with a period of 12
    df['month_sin'] = sin_transformer(12).fit_transform(df['month'])
    df['month_cos'] = cos_transformer(12).fit_transform(df['month'])

    # encode the season with a period of 4
    df['season_sin'] = sin_transformer(4).fit_transform(df['season'])
    df['season_cos'] = cos_transformer(4).fit_transform(df['season'])
    
#     # encode the quarter with a period of 4
#     df['quarter_sin'] = sin_transformer(4).fit_transform(df['quarter'])
#     df['quarter_cos'] = cos_transformer(4).fit_transform(df['quarter'])

In [20]:
#Encode start date
train['DATE'] = pd.to_datetime(train['startdate'])
test['DATE'] = pd.to_datetime(test['startdate'])

convert_time(train)
add_season(train)

convert_time(test)
add_season(test)

encode_cyclical(train)
encode_cyclical(test)

In [21]:
train.head(10)

,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,year,month,day,season,day_of_year_sin,day_of_year_cos,month_sin,month_cos,season_sin,season_cos
0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,244,3,-0.871706,-0.490029,-1.0,-1.836970e-16,-1.0,-1.836970e-16
1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,245,3,-0.880012,-0.474951,-1.0,-1.836970e-16,-1.0,-1.836970e-16
2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,246,3,-0.888057,-0.459733,-1.0,-1.836970e-16,-1.0,-1.836970e-16
3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,247,3,-0.895839,-0.444378,-1.0,-1.836970e-16,-1.0,-1.836970e-16
4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,248,3,-0.903356,-0.428892,-1.0,-1.836970e-16,-1.0,-1.836970e-16
5,0.0,0.833333,9/6/14,237.87,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,249,3,-0.910605,-0.413279,-1.0,-1.836970e-16,-1.0,-1.836970e-16
6,0.0,0.833333,9/7/14,236.36,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,250,3,-0.917584,-0.397543,-1.0,-1.836970e-16,-1.0,-1.836970e-16
7,0.0,0.833333,9/8/14,233.36,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,251,3,-0.924291,-0.381689,-1.0,-1.836970e-16,-1.0,-1.836970e-16
8,0.0,0.833333,9/9/14,233.82,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,252,3,-0.930724,-0.365723,-1.0,-1.836970e-16,-1.0,-1.836970e-16
9,0.0,0.833333,9/10/14,229.74,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,253,3,-0.936881,-0.349647,-1.0,-1.836970e-16,-1.0,-1.836970e-16


### REMOVE OUTLINER

In [22]:
# x = remove_outliers(x, relevant_features[3:-1], 4)

In [23]:
#Split x, y 
target = 'contest-tmp2m-14d__tmp2m'
drop_cols = ['DATE', 'startdate', target]
y = train[target]
X = train.drop([x for x in drop_cols if x in train.columns], axis=1)
X_test = test.drop([x for x in drop_cols if x in test.columns], axis=1)

In [24]:
#defining metric
def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

In [25]:
X.columns.tolist()

['lat',
 'lon',
 'contest-pevpr-sfc-gauss-14d__pevpr',
 'nmme0-tmp2m-34w__cancm30',
 'nmme0-tmp2m-34w__cancm40',
 'nmme0-tmp2m-34w__ccsm30',
 'nmme0-tmp2m-34w__ccsm40',
 'nmme0-tmp2m-34w__cfsv20',
 'nmme0-tmp2m-34w__gfdlflora0',
 'nmme0-tmp2m-34w__gfdlflorb0',
 'nmme0-tmp2m-34w__gfdl0',
 'nmme0-tmp2m-34w__nasa0',
 'nmme0-tmp2m-34w__nmme0mean',
 'contest-wind-h10-14d__wind-hgt-10',
 'nmme-tmp2m-56w__cancm3',
 'nmme-tmp2m-56w__cancm4',
 'nmme-tmp2m-56w__ccsm3',
 'nmme-tmp2m-56w__ccsm4',
 'nmme-tmp2m-56w__cfsv2',
 'nmme-tmp2m-56w__gfdl',
 'nmme-tmp2m-56w__gfdlflora',
 'nmme-tmp2m-56w__gfdlflorb',
 'nmme-tmp2m-56w__nasa',
 'nmme-tmp2m-56w__nmmemean',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__cancm3',
 'nmme-prate-34w__cancm4',
 'nmme-prate-34w__ccsm3',
 'nmme-prate-34w__ccsm4',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__gfdl',
 'nmme-prate-34w__gfdlflora',
 'nmme-prate-34w__gfdlflorb',
 'nmme-prate-34w__nasa',
 'nmme-prate-34w__nmmemean',
 'contest-wind-h100-14d__wind-hgt-100',


# Feature Selection based on Random Forest Regressor

In [26]:
!pip install sklearn

In [27]:
# from sklearn.feature_selection import SelectFromModel
# sel = SelectFromModel(RandomForestRegressor(n_estimators=300))
# sel.fit(X, y)

In [28]:
# sel.get_support()
# selected_feat= X.columns[(sel.get_support())]
# len(selected_feat)

# Modeling & Cross Validation 

In [29]:
#Train tesst
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)
print(f'Train_shape: {X_train.shape}    |   Val_shape: {X_val.shape}    |   Test_shape: {X_test.shape}')

Train_shape: (251741, 253)    |   Val_shape: (123993, 253)    |   Test_shape: (31354, 253)


In [ ]:
#pip install catboost

In [ ]:
#pip install xgboost

# Use Lightgbm

In [ ]:
#pip install lightgbm

In [ ]:
import lightgbm as lgb

print("Beginning training and fitting lightgbm model")
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'max_depth': 8,
    'learning_rate': 0.03,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 50,
    'min_data_in_leaf': 100,
    'n_estimators': 15000,
}

# Create the LightGBM model object
reg_lgb = lgb.LGBMRegressor(**params)

# Fit the model to the training data
reg_lgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)])


# Get feature importances
importance_scores = reg_lgb.feature_importances_
feature_importances = pd.DataFrame({'feature': X_train.columns, 'importance': importance_scores})

# Sort the features by importance score
feature_importances = feature_importances.sort_values('importance', ascending=False)

In [ ]:
importance

In [ ]:
# Generate predictions on pseudo-eval set (splitted from train set)
# Create the LightGBM model object
# Fit the model to the training data
reg_lgb_1.fit((X, y))

In [ ]:
#select features with recursive feature elimination
from sklearn.feature_selection import RFE
rfe = RFE(reg_lgb, n_features_to_select=50, step=1)
rfe.fit(X_train, y_train)
print(rfe.support_)

# summarize the ranking of the attributes
fea_rank_ = pd.DataFrame({'cols':X_train.columns,
                          'fea_rank':rfe.ranking_})

In [ ]:
#train model on full data
y_pred_lgb_1 = reg_lgb_1.predict(X_test)

In [ ]:
# submission = pd.read_csv('sample_solution.csv')
# submission[target] = y_pred_lgb_1
# submission.to_csv('submission1_lgb.csv', 
#                   index = False)

## Use MLP

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
params = (hidden_layer_sizes=(150,100,50),
                       max_iter = 300,activation = 'relu',
                       solver = 'adam')

# Standardize the data using a standard scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

reg_mlp = MLPRegressor(**params)

reg_mlp.fit(X_train, y_train)

# make predictions on the test data
y_pred_mlp = reg_mlp.predict(X_val)

In [ ]:
#Run feature selection by RFR
from sklearn.feature_selection import SelectFromModel
sel = SelectFromModel(RandomForestRegressor(n_estimators=200))
sel.fit(X, y)
sel.get_support()
selected_feat= X.columns[(sel.get_support())]
print(len(selected_feat), '/n', selected_feat)

# Ensembling prediction

In [34]:
import xgboost as xgb

reg_xgb = xgb.XGBRegressor(base_score=0.5, 
                           n_estimators=1500, 
                           objective='reg:squarederror',
                           max_depth=5,
                           early_stopping_rounds=20,
                           learning_rate=0.03)

reg_xgb.fit(X_train, y_train, eval_set=((X_val, y_val),), verbose=100)

[0]	validation_0-rmse:14.59301
[100]	validation_0-rmse:1.62799
[200]	validation_0-rmse:1.12891
[300]	validation_0-rmse:1.01038
[400]	validation_0-rmse:0.92661
[500]	validation_0-rmse:0.86679
[600]	validation_0-rmse:0.81871
[700]	validation_0-rmse:0.78014
[800]	validation_0-rmse:0.75023
[900]	validation_0-rmse:0.72192
[1000]	validation_0-rmse:0.70009
[1100]	validation_0-rmse:0.67788
[1200]	validation_0-rmse:0.66007
[1300]	validation_0-rmse:0.64330
[1400]	validation_0-rmse:0.62719
[1499]	validation_0-rmse:0.61386


XGBRegressor(base_score=0.5, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=20,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.03, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

# Hyperparameter tuning

# Train on full data

#Submission

#Reference